In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from numpy import prod
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, concatenate, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

NMS_THRESHOLD=0.3
MIN_CONFIDENCE=0.2

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPU 2 to use

In [2]:
def pedestrian_detection(image, model, layer_name, personidz=0):
	(H, W) = image.shape[:2]
	results = []


	blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	model.setInput(blob)
	layerOutputs = model.forward(layer_name)

	boxes = []
	centroids = []
	confidences = []

	for output in layerOutputs:
		for detection in output:

			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			if classID == personidz and confidence > MIN_CONFIDENCE:

				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))
	# apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
	idzs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONFIDENCE, NMS_THRESHOLD)
	# ensure at least one detection exists
	if len(idzs) > 0:
		# loop over the indexes we are keeping
		for i in idzs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])
			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			res = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(res)
	# return the list of results
	return results

In [3]:
#location = new_datasets/train/1
#file_name = features_train_fight
def preprocess(location, save_path, file_name):
    
    print(IMG_SIZE)
    base = tf.keras.applications.MobileNetV3Small(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet', include_top=False)
    features = np.empty((0, 28224))
    labels = []
    base_model = tf.keras.Sequential([
      base,
      Flatten()
    ])
    base_model.summary()
    base_model.trainable = False
    index = 0
    label = location[-1]
    labelsPath = "coco.names"
    LABELS = open(labelsPath).read().strip().split("\n")

    weights_path = "yolov4-tiny.weights"
    config_path = "yolov4-tiny.cfg"

    model = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    layer_name = model.getLayerNames()
    layer_name = [layer_name[i - 1] for i in model.getUnconnectedOutLayers()]
    writer = None
    
    
    
    for lb, i in enumerate(os.listdir(location)): #location = new_datasets/train/1
        filedir = os.path.join(location, i)
        for j in os.listdir(filedir):
            split_video = os.path.join(filedir, j)
            frames = np.array(list(map(lambda x: os.path.join(split_video, x), os.listdir(split_video))))
            frames = sorted(frames, key=lambda x: int(x.split("/")[-1].strip(".jpg")))
            if len(frames) > 3000:
                print("frames over : ", len(frames))
                n = len(frames) - 3000
                n = n//2
                frames = frames[n:-n]
                
#             cutline = len(frames) % LSTM_INTERVAL
#             cutline -= 1
#             if cutline:
#                 frames = frames[:-cutline]
            
#             if os.path.isfile(save_path+'/'+file_name+'_'+str(index)+'.npy'):
#                 index+=len(frames)// (LSTM_INTERVAL+1)
#                 continue

            labels.append([label]*(len(frames)))

            frames = np.array(list(map(lambda x:  cv2.resize(cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB),(IMG_SIZE, IMG_SIZE)), frames)))
            #             frames = np.array(list(map(lambda x: cv2.resize(x, (IMG_SIZE, IMG_SIZE)), frames)))
    #             if len(frames) > 5000:
    #                 n = len(frames) - 5000
    #                 n = n//2
    #                 frames = frames[:]
            

            if len(frames) < (LSTM_INTERVAL + 1):
                continue
            
            mask_list = []
            for frame in frames:
                results = pedestrian_detection(frame, model, layer_name,personidz=LABELS.index("person"))
                mask = np.zeros(frame.shape[:2], np.uint8)
                for res in results:
                    polygon = np.array([[res[1][0]-10,res[1][1]-10],[res[1][2]+10,res[1][1]-10],[res[1][2]+10,res[1][3]+10],[res[1][0]-10,res[1][3]+10]])
                    cv2.fillPoly(mask,[polygon],1)
#                 plt.figure(figsize=(20,20))
#                 plt.imshow(frame)
#                 plt.show()
                mask_list.append(mask.astype(bool))
#                frame = frame * mask[:, :, np.newaxis]
#                 temp_frames.append(frame)
                

            temp_frames = []
                                                           
#             print("len frames: ", len(frames))
            for k in range(1, len(frames)):
                if os.path.isfile(save_path+'/'+file_name+'_'+str(index)+'.npy'):
                    index+=1
                    continue
                hsv = np.zeros_like(frames[k])
                hsv[...,1] = 255
                dst1 = frames[k-1]
                dst2 = frames[k]                         
                img_1 = cv2.cvtColor(dst1,cv2.COLOR_BGR2GRAY)
                img_2 = cv2.cvtColor(dst2,cv2.COLOR_BGR2GRAY)
                flow = cv2.calcOpticalFlowFarneback(img_1, img_2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                hsv[...,0] = ang*180/np.pi/2
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
#                 plt.figure(figsize=(20,20))
#                 plt.imshow(rgb)
#                 plt.show()
                rgb = rgb * (mask_list[k-1][:, :, np.newaxis] + mask_list[k][:, :, np.newaxis])
#                 plt.figure(figsize=(20,20))
#                 plt.imshow(rgb)
#                 plt.show()
                temp_frames.append(rgb)
            frames = np.array(temp_frames)
#             print("len frames 2: ", len(frames))
            feature_list = base_model.predict(frames)
            for ind in range(0, len(feature_list), LSTM_INTERVAL):
                np.save(save_path+'/'+file_name+'_'+str(index)+'.npy',feature_list[ind:ind+LSTM_INTERVAL])


                index+=1
        print("doing", index)
#         if index > 6000:
#             break
    print("done")
            #temp = np.concatenate([temp, feature_list])
#             if temp.shape[0] > 5000:
#                 features = np.concatenate([features, temp])
#                 temp = np.empty((0, 28224))
             #   print(features.shape)
    #if temp.shape[0] >0:
    #    features = np.concatenate([features, temp])
    #print(features.shape)
#     labels = sum(labels, [])
#     return features, labels

In [4]:
LSTM_INTERVAL = 30
IMG_SIZE = 224

In [6]:
location = "원본/dataset/train/0"
save_path = "fix_train_data/0/"
file_name = 'features_train_fight2'
preprocess(location, save_path, file_name)

224
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten_1 (Flatten)         (None, 28224)             0         
                                                                 
Total params: 939,120
Trainable params: 927,008
Non-trainable params: 12,112
_________________________________________________________________
frames over :  5157
frames over :  3408
frames over :  3825
frames over :  3709
frames over :  4129
frames over :  3883
frames over :  5187
frames over :  4838
frames over :  3249
frames over :  3166
frames over :  4349
frames over :  3005
frames over :  3904
frames over :  4306
frames over :  4004
frames over :  3411
frames over :  4255
doing 74
done


In [7]:
location = "원본/dataset/train/2"
save_path = "fix_train_data/2/"
file_name = 'features_train_nomal2'
preprocess(location, save_path, file_name)

224
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten_2 (Flatten)         (None, 28224)             0         
                                                                 
Total params: 939,120
Trainable params: 927,008
Non-trainable params: 12,112
_________________________________________________________________
frames over :  8926
frames over :  4162
frames over :  9163
frames over :  9036
frames over :  8623
frames over :  9060
frames over :  8444
frames over :  9157
frames over :  7081
frames over :  5934
frames over :  7563
frames over :  5391
frames over :  4310
frames over :  5588
frames over :  9239
frames over :  8069
frames over :  7311
frames over :  8548
frames ov

Premature end of JPEG file


error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
location = "원본/dataset/val/2"
save_path = "/home/nplab/Jueon/LSTM/datasets/val/2"
file_name = 'features_val_normal2'
preprocess(location, save_path, file_name)

224


2022-06-21 01:48:44.502305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 01:48:44.523217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 01:48:44.523451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 01:48:44.523983: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 28224)             0         
                                                                 
Total params: 939,120
Trainable params: 927,008
Non-trainable params: 12,112
_________________________________________________________________
frames over :  8873


In [ ]:
location = "원본/dataset/val/0"
save_path = "/home/nplab/Jueon/LSTM/datasets/val/0"
file_name = 'features_train_fight2'
preprocess(location, save_path, file_name)

In [ ]:
location = "원본/dataset/train/2"
save_path = "/home/nplab/Jueon/LSTM/datasets/train/2"
file_name = 'features_val_normal2'
preprocess(location, save_path, file_name)

In [ ]:
location = "원본/dataset/train/0"
save_path = "/home/nplab/Jueon/LSTM/datasets/train/0"
file_name = 'features_train_fight2'
preprocess(location, save_path, file_name)